In [1]:
# Importing important libraries 
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
import pandas as pd

/Users/bhuyash./Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
news_count_threshold = 5000
news_count = 0

In [3]:
sub_cat_dict = {"society": ["politics", "crime", "fact-check"], 
                "business": ["markets", "companies", "finance"], 
                "technology": ["smartphones", "tablets", "internet", "games"], 
                "sports": ["indian-premier-league", "football", "cricket"], 
                "entertainment": ["bollywood", "hollywood", "television", "south-cinema"]}

In [4]:
total_cat_count = len(sub_cat_dict)
total_sub_cat_count = len([i for v in sub_cat_dict.values() for i in v])
news_per_page_count = 15

In [5]:
articles_per_sub_cat = news_per_page_count*total_sub_cat_count
if news_count_threshold > articles_per_sub_cat:
    pages_per_sub_cat = int(round(news_count_threshold/articles_per_sub_cat))
elif news_count_threshold <= articles_per_sub_cat:
    pages_per_sub_cat = 2

In [6]:
news_headlines = list()
news_links = list()
news_summaries = list()
news_authors = list()
news_dates = list()
news_times = list()
news_entire = list()
news_cats = list()
news_subcats = list()


for cat in sub_cat_dict:
    for sub_cat in sub_cat_dict[cat]:
        for page_no in range(1, pages_per_sub_cat):
            if news_count <= news_count_threshold:
                # Specifying the URL 
                url = "https://www.ibtimes.co.in/{}/page/{}".format(sub_cat, page_no)
                # Since the request can raise error
                try:
                    page = requests.get(url)
                    if page.status_code != 200:
                        print(page.status_code)
                        continue

                    # Using BeautifulSoup for web crawling
                    soup = BeautifulSoup(page.content, "html.parser")

                    # Fetching the child of type <class 'bs4.element.Tag'> or HTML in other words
                    for i in list(soup.children):
                        if str(type(i)) == "<class 'bs4.element.Tag'>":
                            html_file = i

                    articles = html_file.find_all("article")[:15]

                    for article in articles:
                        headline = article.find_all("h2")[0].get_text()
                        news_link = article.find_all("a")[1]["href"]
                        summary = article.find_all("summary")[0].get_text()
                        try:
                            news_page = requests.get(news_link)
                            if news_page.status_code != 200:
                                print(news_page.status_code)
                                continue

                            # Using BeautifulSoup for web crawling
                            news_soup = BeautifulSoup(news_page.content, "html.parser")

                            # Fetching the child of type <class 'bs4.element.Tag'> or HTML in other words
                            for i in list(news_soup.children):
                                if str(type(i)) == "<class 'bs4.element.Tag'>":
                                    news_html_file = i

                            author = news_html_file.find_all("span", class_ = "author")[0].get_text().strip()
                            date = " ".join(news_html_file.find_all("time")[0].get_text().strip().split()[:3])
                            time = " ".join(news_html_file.find_all("time")[0].get_text().strip().split()[3:])
                            entire_news = ""
                            for p in news_html_file.find_all("p"):
                                entire_news += p.get_text().strip()

                            news_headlines.append(headline)
                            news_links.append(news_link)
                            news_summaries.append(summary)
                            news_authors.append(author)
                            news_dates.append(date)
                            news_times.append(time)
                            news_entire.append(entire_news)
                            news_cats.append(cat)
                            news_subcats.append(sub_cat)
                            
                            headline, news_link, summary, author, date, time, entire_news = "", "", "", "", "", "", ""
                            
                            news_count += 1
                            
                            if news_count >= news_count_threshold:
                                break

                        except Exception as e:
                            error_type, error_obj, error_info = sys.exc_info()
                            print("URL: ", news_link)
                            print(error_type, "Line: ", error_info.tb_lineno)
                            continue


                except Exception as e:
                    error_type, error_obj, error_info = sys.exc_info()
                    print("URL: ", url)
                    print(error_type, "Line: ", error_info.tb_lineno)
                    continue

KeyboardInterrupt: 

In [7]:
News_Corpus = pd.DataFrame({"Date": news_dates, "Time": news_times, "Category": news_cats,
                            "Sub-Category": news_subcats, "Headline": news_headlines, "Summary": news_summaries,
                           "Entire_News": news_entire, "Author": news_authors, "News_Link": news_links})

In [8]:
News_Corpus

,Date,Time,Category,Sub-Category,Headline,Summary,Entire_News,Author,News_Link
0,"March 22, 2021",12:23 IST,society,politics,"After ripped jeans row, Uttarakhand CM makes a...",Uttarakhand Chief Minister Tirath Singh Rawat ...,Uttarakhand Chief Minister Tirath Singh Rawat ...,Soorya Kiran NN,https://www.ibtimes.co.in/after-ripped-jeans-r...
1,"March 21, 2021",14:51 IST,society,politics,Resign Anil Deshmukh trends as pressure mounts...,Sources within the alliance say that allegatio...,There is a pressure building within the allian...,IBT News Desk,https://www.ibtimes.co.in/resign-anil-deshmukh...
2,"March 21, 2021",14:42 IST,society,politics,"At whose behest SUV was parked near Antilia, a...",The MNS chief said raised questions on the nex...,A day after demanding the resignation of state...,IBT News Desk,https://www.ibtimes.co.in/whose-behest-suv-was...
3,"March 20, 2021",21:20 IST,society,politics,Jubilee Hills Housing Society: Full page adver...,"The nomination process began last month, amids...",Two full pages on the front of major daily new...,Manpriya Khurana,https://www.ibtimes.co.in/jubilee-hills-housin...
4,"March 18, 2021",18:55 IST,society,politics,"Onscreen Lord Ram joins BJP, three decades aft...","Last year, amidst lockdown, Doordarshan once a...",The onscreen Lord Ram -- Arun Govil -- joined ...,Manpriya Khurana,https://www.ibtimes.co.in/arun-govil-tv-serial...
...,...,...,...,...,...,...,...,...,...
4781,"September 23, 2020",17:10 IST,entertainment,south-cinema,"Yash bulking up muscles, on a strict diet as p...",Rocking Star has dooubled his workout regime a...,Rocking Star Yash is leaving no stone unturned...,David Johnson,https://www.ibtimes.co.in/yash-bulking-muscles...
4782,"September 23, 2020",15:25 IST,entertainment,south-cinema,"Rajinikanth hesitant, KS Ravikumar interested ...",KS Ravikumar has revealed his meeting with Raj...,Rajinikanth and KS Ravikumar were keen to do a...,David Johnson,https://www.ibtimes.co.in/rajinikanth-hesitant...
4783,"September 22, 2020",16:59 IST,entertainment,south-cinema,GVP on Ajith-Sudha Kongara project: It has fan...,GV Prakash Kumar speaks about Sudha Kongara's ...,There have been lots of rumours about Sudha Ko...,David Johnson,https://www.ibtimes.co.in/gvp-ajith-sudha-kong...
4784,"September 22, 2020",11:51 IST,entertainment,south-cinema,Kasturi: Allegations of sexual assault without...,Actress Kasturi Shankar has expressed her view...,Kasturi Shankar has responded to the sexual ha...,David Johnson,https://www.ibtimes.co.in/kasturi-allegations-...


In [9]:
News_Corpus.to_csv("MS18034_Abhishek_Kumar_Singh_22-03-2021.csv", index = False)